In [ ]:
def leading_bin(x,length):
    return bin(x)[2:] if len(bin(x)[2:])>=length else "0"*(length-len(bin(x)[2:])) + bin(x)[2:]
def count_ones(word):
    ret=0
    for i in word:
        ret+= 1 if int(i)==1 else 0
    return ret
def invert(word):
    return ''.join(['1' if i=='0' else '0' for i in word])

name5={leading_bin(i,5):"D%02d"%i for i in range(32)}
name3={leading_bin(i,3):".%d"%i for i in range(8)}

#codes that are mirrored for RB+ and RB-
d5n={"00000":"100111", "00001":"011101", "00010":"101101",
     "00100":"110101", "00111":"111000", "01000":"111001",
     "01111":"010111", "10000":"011011", "10111":"111010",
     "11000":"110011", "11011":"110110", "11101":"101110",
     "11110":"011110", "11111":"101011"}
#codes that are NOT mirrored for RB+ and RB-
d5c={"00011":"110001", "00101":"101001", "00110":"011001",
     "01001":"100101", "01010":"010101", "01011":"110100",
     "01100":"001101", "01101":"101100", "01110":"011100",
     "10001":"100011", "10010":"010011", "10011":"110010",
     "10100":"001011", "10101":"101010", "10110":"011010",
     "11001":"100110", "11010":"010110", "11100":"001110",}

#codes that are mirrored for RB+ and RB-
d3n={"000":"1011", "011":"1100", "100":"1101",}
#codes that are NOT mirrored for RB+ and RB-
d3c={"001":"1001", "010":"0101", "101":"1010", "110":"0110",}

# Code 7 from d3 is an exception: For data, d3p7n needs be used, 
# unless for D.17 D.18 D.20 in RB- and unless for D.11 D.13 D.14 in RB+. There, d3a7n needs to be used
# All control codes use d3a7n (implicitely)
d3p7n={"111":"1110"}
d3a7n={"111":"0111"}

# Control codes are always mirrored.
k3n={"000":"1011", "001":"0110", "010":"1010", "011":"1100",
     "100":"1101", "101":"0101", "110":"1001", "111":"0111",}
k5n={k:v for k,v in d5n.items()}
k5n["11100"]="001111"
k5c=d5c
def lut_8b10b(val8):
    val=leading_bin(val8,8)
    high = val[:3]
    low = val[3:]
    
    if low in d5n:
        low6n = d5n[low]
        low6p = invert(d5n[low])
    else:
        low6n = d5c[low]
        low6p = d5c[low]
    should_invert=count_ones(low6n)>3
    if high =="111":
        if int(low,2) in [17,18,20]:
            high4n = invert(d3a7n[high]) if should_invert else d3a7n[high]
            assert should_invert==False,"Strange?"
        else:
            high4n = invert(d3p7n[high]) if should_invert else d3p7n[high]
                
        if int(low,2) in [11,13,14]:    
            high4p = d3a7n[high]  if should_invert else invert(d3a7n[high])
            assert should_invert==False,"Strange?"
        else:
            high4p = d3p7n[high]  if should_invert else invert(d3p7n[high])    
    elif high in d3n:
        high4n = invert(d3n[high]) if should_invert else d3n[high]
        high4p = d3n[high] if should_invert else invert(d3n[high])
    else:
        high4n = d3c[high]
        high4p = d3c[high]
        
    #print name5[low]+name3[high], low6n+" "+high4n,"   ",low6p+" "+high4p
    return name5[low]+name3[high], low6n+high4n,low6p+high4p

def lut_8b10b_control(val8):
    val=leading_bin(val8,8)
    high = val[:3]
    low = val[3:]
    if low in k5n:
        low6n = k5n[low]
        low6p = invert(k5n[low])
    else:
        low6n = k5c[low]
        low6p = k5c[low]
    should_invert=count_ones(low6n)>3
    high4n = invert(k3n[high]) if should_invert else k3n[high]
    high4p = k3n[high] if should_invert else invert(k3n[high])

    #print name5[low].replace("D","K")+name3[high], low6n+" "+high4n,"   ",low6p+" "+high4p
    return name5[low].replace("D","K")+name3[high], low6n+high4n,low6p+high4p


final_lut={}
final_lut8={}
for i in range(256):
    x=lut_8b10b(i)
    final_lut8[x[0]]=i
    if x[1]==x[2]:
        final_lut[x[1]]=x[0]+"o"
    else:
        final_lut[x[1]]=x[0]+"-"
        final_lut[x[2]]=x[0]+"+"

for i in [28,60,92,124,156,188,220,252,247,251,253,254]:
    x=lut_8b10b_control(i)
    final_lut[x[1]]=x[0]+"-"
    final_lut[x[2]]=x[0]+"+"
    
final_lut8["ERROR"]=None
for i in range(1024):
    if leading_bin(i,10) not in final_lut:
        final_lut[leading_bin(i,10)]="ERROR "

The following Cell does contain the the class which translates the oszi data to a bitpattern.

In [ ]:
import time
import numpy as np
from blist import blist
import pandas as pd
class SignalClass():
    def __init__(self, filename):
        self._data = None
        self._filename = filename
        self._triggerPoint = 0

    def saveData(self,filename):
        Time = self._data[:, 0]
        Amplitude = self._data[:, 1]
        BitString = self._bitvalues
        TriggerStart = int(self._triggerPoint / self._bitwidth)
        BitCenterIndex = self._central
        np.savez(filename, Time=Time, Amplitude=Amplitude, BitString=BitString, TriggerStart=TriggerStart,BitCenterIndex=BitCenterIndex)

    def loadData(self,filename):
        d = np.load(filename)
        Time = d['Time']
        Amplitude = d['Amplitude']
        BitString = d['BitString']
        TriggerStart = d['TriggerStart']
        return Time, Amplitude, BitString, TriggerStart

    def loadRealWaveform(self):
        length = 0
        print "start loading"
        data = pd.read_csv(self._filename, delimiter = ",", skiprows = 5, usecols = (3, 4), index_col = False).values
        print "loading done. shape", data.shape
        # data[timeID] = (time,value)
        self._data = np.array(data)
        self._triggerPoint = np.where(self._data[:, 0] < 0.)[0][-1]
        # print self._triggerPoint
        return self._data

    def getBitString(self):
        print "getBitString"
        bwticks, transIdx, ctrIdx, bitValue, cm = self.getBitWidthTicks(self._data[:, 1])
        self._bitwidth = int(bwticks)
        # print transIdx, bwticks
        N = int(len(self._data) / bwticks)
        self._transition, self._central, self._bitvalues = self.fillUp(transIdx, bitValue, bwticks, N)
        return self._bitvalues

    def getBitWidthTicks(self, dataIn):
        cnt = 0
        lastVal = dataIn[0]
        bitWidthList = []
        transitionIdx = []
        bitValue = []
        centralIdx = []
        cm = np.mean(dataIn)
        print "Parsing data..."
        for dix, d in enumerate(dataIn):
            if (d - cm) * (lastVal - cm) >= 0.:
                cnt = cnt + 1
            else:
                bitWidthList.append(cnt)
                transitionIdx.append(int(dix))
                if d < cm:
                    bitValue.append(0)
                else:
                    bitValue.append(1)
                centralIdx.append(int(np.round(dix + cnt / 2.)))
                cnt = 0
            lastVal = d
        bitWidthList.sort()

        subList, N = self.find_sub_list(bitWidthList)
        # delete strange values.
        max_len=0;
        for l in subList:
            max_len=max(len(l),max_len)
        subList=[l for l in subList if len(l)>1+max_len/1000] # I don't remember why this is here...
        subList=[l for l in subList if l[0]>10]  # Very short values are possibly jitter.
        bitWidth = 0.0
        meanOfOneBit = np.mean(subList[0])
        print "A bit seems to be ",round(meanOfOneBit,1)," samples long"
        for val in subList:
            m = np.mean(val)
            div = int(round(m / meanOfOneBit))
            weightedTerm = m * len(val) / div
            bitWidth = bitWidth + weightedTerm
        # print transitionIdx
        print "done"
        return bitWidth / N, transitionIdx, centralIdx, bitValue, cm

    def fillUp(self, data, data2, N, nbits):
        d = blist(data)
        d1 = []
        d2 = blist(data2)

        n = 1
        print "filling up seqences of multiple same bits"
        while (n < nbits):  # and (len(d) < n):
            try:
                if (d[n] - d[n - 1]) > 1.2 * N:
                    d.insert(n, int(np.round(d[n - 1] + N)))
                    d2.insert(n, int(d2[n - 1]))
                elif (d[n] - d[n - 1]) < 0.2 * N:
                    del d[n]
                    del d2[n]
                    continue
            except IndexError as e:
                pass
            n = n + 1
        return np.array(d,dtype=int), np.array(d,dtype=int)+int(N/2), np.array(d2,dtype=int)

    def find_sub_list(self, inList):
        outList = []
        subList = []
        lastVal = inList[0]
        numberOfEntries = 0
        for val in inList:
            if (val - lastVal <= 0.5 * inList[0]):  # and (val > 60):
                subList.append(val)
            else:
                if len(subList) > 0:
                    # print subList
                    outList.append(subList)
                    numberOfEntries = numberOfEntries + len(subList)
                    subList = []
                    subList.append(val)
                    lastVal = val
        if len(outList) == 0:
            outList = [inList]
            numberOfEntries = len(inList)
        return outList, numberOfEntries

Load the data. Here you have to put your file names

In [ ]:
filename = "D:\\Arbeit\\hslink\\decode\\hs16gates1_hs.csv"
signal = SignalClass(filename)
signal.loadRealWaveform()
bstr = signal.getBitString()
signal.saveData("D:\\Arbeit\\hslink\\HSDecodeData.npz")
del signal

In [ ]:
filename = "D:\\Arbeit\\hslink\\decode\\hs16gates1_triggerline.csv"
signal = SignalClass(filename)
signal.loadRealWaveform()
bstr = signal.getBitString()
signal.saveData("D:\\Arbeit\\hslink\\triggerDecodeData.npz")
del signal

The following code decodes the trigger line. It searches for a broken manchester code  (00011101) and synchronizes for it. Currently, there is no handle for a bad polarity.

Then, a plot of the data is created...



In [ ]:
manchester={"01":0,"10":1}
offset_trigger=trigger_bits.find("00011101")
triggerTimeVect=b["Time"]
triggerBitCenterIndex=b["BitCenterIndex"]


t=[ triggerTimeVect[triggerBitCenterIndex[i]] for i in range(offset_trigger,len(trigger_bits)-7,8)]
t.append( triggerTimeVect[triggerBitCenterIndex[-1]])                                             
trigger_symbol_time=np.asarray(t)


triggersymbols=[ trigger_bits[i:(i+8)] for i in range(offset_trigger,len(trigger_bits)-7,8)]
trigger_curve={"memdump":[],"fsync":[],"trigger":[],"veto":[],"reset":[],"time":[]}
for idx,sym in enumerate(triggersymbols):
    for i in range(2):
        trigger_curve["memdump"].append(1 if sym.startswith("111000") else 0)
        trigger_curve["fsync"].append(manchester[sym[6:8]])
        trigger_curve["trigger"].append(manchester[sym[4:6]] if not (sym.startswith("111000") or sym.startswith("000111")) else 0)
        trigger_curve["veto"].append(manchester[sym[2:4]] if not (sym.startswith("111000") or sym.startswith("000111")) else 0)
        trigger_curve["reset"].append(manchester[sym[0:2]] if not (sym.startswith("111000") or sym.startswith("000111")) else 0)
    trigger_curve["time"].append(trigger_symbol_time[idx])
    trigger_curve["time"].append(trigger_symbol_time[idx+1])
                                                   
trigger_curve={k:np.array(v) for k,v in trigger_curve.items()}
  
memdump=[1 if sym.startswith("111000") else 0 for sym in triggersymbols]
fsync=[manchester[sym[6:8]] for sym in triggersymbols]
trigger=[manchester[sym[4:6]] if not (sym.startswith("111000") or sym.startswith("000111")) else 0 for sym in triggersymbols]
veto=[manchester[sym[2:4]] if not (sym.startswith("111000") or sym.startswith("000111")) else 0 for sym in triggersymbols]
reset=[manchester[sym[0:2]] if not (sym.startswith("111000") or sym.startswith("000111")) else 0 for sym in triggersymbols]
from matplotlib import pyplot as pl
pl.figure(figsize=(12,5),dpi=180)
pl.plot(trigger_curve["time"],trigger_curve["memdump"]+2.2)
pl.plot(trigger_curve["time"],trigger_curve["fsync"]+1.1)
pl.plot(trigger_curve["time"],trigger_curve["trigger"])
pl.plot(trigger_curve["time"],trigger_curve["veto"]-1.1)
pl.plot(trigger_curve["time"],trigger_curve["reset"]-2.2)
pl.show()

In [ ]:
import numpy as np
a=np.load("D:\\Arbeit\\hslink\\HSDecodeData.npz")
s="".join(map(str,a["BitString"]))

b=np.load("D:\\Arbeit\\hslink\\triggerDecodeData.npz")
trigger_bits="".join(map(str,b["BitString"]))
print "Data bits", len(s), "Trigger bits", len(trigger_bits)

We don't know the polarity of the data.
Startpoint:
We know that we have 5 consecutive zeros or ones in some data word at a special place. This can be used to determine the offset.

Next, we need the polarity. There is no way to get it from the data immediatelly.
BUT we know how our data should look like. It should start with the bits '101' or '000'. We translate both ways and have a look which one is correct. 

Another way would be to use the commented section which tries every combination.

In [ ]:
offset=None
symbols=[]
# for offset_test in range(10):
#     for i in range(offset_test,len(s)-9,10):
#         if s[i:(i+10)] not in final_lut:
#             print "offset",offset_test, "does not seem to work.",s[i:(i+10)],"is not a valid word. Correct words:",i/10
#             break
#     else:
#         print "offset",offset_test, " works. All words correct!"
#         offset=offset_test

idx=s.find("11111")  
idx2=s.find("00000")
assert (idx-2)%10 == (idx2-2)%10,"00000 and 11111 suggest different offset!"   
print "found sequence of 5 consecutive symbols at ", idx, "and",idx2,"! Offset: ",(idx-2)%10
offset=(idx-2)%10
     
symbols=[ final_lut[s[i:(i+10)]] for i in range(offset,len(s)-9,10)]  #  with positive, negative
symbol=[x[:-1] for x in symbols]  #  without positive, negative
try:
    start_idx=symbol.index('K28.2')   # Data start
    assert symbol[start_idx+1]=='K27.7' # Data start2
    w1=final_lut8[symbol[start_idx+2]]  # first data word. start2
    inv=final_lut[invert(s[(10*(start_idx+2)+offset):(10*(start_idx+2)+10+offset)])]
    w2=final_lut8[inv[:-1]]  # first data word. start2
    if bin(w1)[2:5]=="101" or bin(w1)[2:5]=="000":
        print "polarity is correct"
    elif bin(w2)[2:5]=="101" or bin(w2)[2:5]=="000":
        print "polarity is changed"
        s=invert(s)
        symbols=[ final_lut[s[i:(i+10)]] for i in range(offset,len(s)-9,10)]
        symbol=[x[:-1] for x in symbols]
    else:
        raise ValueError("strange first data word %s, %s"%(w1,w2))
except ValueError as e:
        print "could not determine polarity",e
        raise e


In [ ]:
###
# How should special characters be handled? Verbose=True prints comma and other characters,  verbose=false deletes them
###
verbose=False
if verbose:
    special_lut={"K28.5":"/ K /","K28.0":"/ R /","K28.3":"/ A /","K23.7":"/CCC/"}
else:
    special_lut={"K28.5":"","K28.0":"","K28.3":"","K23.7":""}
symbol=[x[:-1] for x in symbols]
symbol=[special_lut[x] if x in special_lut else x for x in symbol]


In [ ]:
frames=[]
idx=0
try:
    while True: 
        start_idx=idx+symbol[idx:].index('K28.2')
        assert symbol[start_idx+1]=='K27.7'
        idx=start_idx+2
        end_idx=idx+symbol[idx:].index('K29.7')
        assert symbol[end_idx+1]=='K30.7'
        idx=end_idx+2
        print "found frame between",start_idx+2,'and',end_idx,' Length:', end_idx-start_idx-2
        frames.append(symbol[start_idx+2:end_idx])
except ValueError:
    pass
for frame in frames:
    print "\n", [ hex(final_lut8[x])[2:] if final_lut8[x] else x for x in frame if (not x=="") and x in final_lut8]
    if len(frame)>4:
        print 1,[ final_lut8[x] for x in frame if (not x=="") and x in final_lut8]
        print 2,[x for x in frame]

Plot the final trigger line data together sith the bit data and its translation

In [ ]:
triggerAmplitude=b["Amplitude"]
amplitude=a["Amplitude"]
time=a["Time"]
bitCentnerIndex=a["BitCenterIndex"]
fig_idx=0
def plot_signals(start_word,end_word,title=""):
    start_bit=offset+10*start_word
    end_bit=offset+10*end_word
    start_index=bitCentnerIndex[start_bit]
    end_index=bitCentnerIndex[end_bit]

    difftime=(time[end_index]-time[start_index])/15
    plot_index=np.logical_and(trigger_curve["time"]>time[start_index],trigger_curve["time"]<time[end_index])
    global fig_idx
    pl.figure(fig_idx,figsize=(12,9),dpi=180)
    fig_idx+=1
    pl.plot(trigger_curve["time"][plot_index],trigger_curve["trigger"][plot_index]*.1-1.07)
    pl.plot(trigger_curve["time"][plot_index],trigger_curve["fsync"][plot_index]*.1-1.2)
    pl.plot(time[start_index:end_index],amplitude[start_index:end_index]+1,color="0.7",lw=0.7)
    pl.plot(time[start_index:end_index],triggerAmplitude[start_index:end_index],lw=0.7)
    pl.text(time[start_index],-1.03,"trigger")
    pl.text(time[start_index],-1.17,"fsync")

    print_symbol=np.logical_and(trigger_symbol_time>time[start_index],trigger_symbol_time<time[end_index])


    for idx,sym in enumerate(triggersymbols):
        if False==print_symbol[idx]:
            continue
        start_time,end_time=trigger_symbol_time[idx],trigger_symbol_time[idx+1]    
        pl.plot([0.95*start_time+0.05*end_time, 0.95*end_time+ 0.05*start_time ],[-0.6,-0.6])

        pl.text(start_time,-0.70,sym,rotation=90,fontsize=8)

    last_t=-1
    last_offset=1.45
    for x in range(start_word,end_word):
        if symbol[x]=="":
            continue

        text_index=bitCentnerIndex[offset+10*x+4]
        s_index=bitCentnerIndex[offset+10*x]
        e_index=bitCentnerIndex[offset+10*x+9]
        pl.plot([time[s_index],time[e_index]],[0.6,0.6])
        if time[text_index]-last_t>difftime:
            last_offset=1.45
            #print "reset", last_t, time[text_index]
            last_t=time[text_index]

        pl.text(time[text_index],last_offset,symbols[x],rotation=0)
        last_offset-=0.05
    pl.ylim([-1.3,1.5]) 

    
start_word=550 
end_word=2132
plot_signals(start_word,end_word)
start_word=0 
end_word=10462
plot_signals(start_word,end_word)

start_word=956 
end_word=1006
plot_signals(start_word,end_word)
pl.show()



